In [1]:
from taxcalc import *
from bokeh.io import show, output_notebook
from bokeh.charts import Bar
import copy
output_notebook()

Loading BokehJS ...

### Missing Variables

In [2]:
cps = pd.read_csv('../taxdata/cps_data/cps.csv')
usable = open('taxcalc/records_variables.json')
usable_vars = json.load(usable)
usable.close()
missing = 0
for item in usable_vars['read'].keys():
    if item not in cps.columns:
        print '{}: {}'.format(item, usable_vars['read'][item]['desc'])
        missing += 1
print '\nTotal Variables Missing: {}'.format(missing)

p23250: Sch D: Net long-term capital gains/losses
p25470: Sch E: Royalty depletion and/or rental depreciation
e09800: Unreported payroll taxes from Form 4137 or 8919
e02000: Sch E rental, royalty, S-corp, etc, income/loss
e62900: Alternative Minimum Tax foreign tax credit from Form 6251
p08000: Other tax credits (but not including Sch R credit)
e58990: Investment income elected amount from Form 4952
e00700: Taxable refunds of state and local income taxes
e03290: Health savings account deduction from Form 8889
e07240: Retirement savings contributions credit from Form 8880
agi_bin: Historical AGI category used in data extrapolation
e27200: Sch E: Farm rent net income or loss
e01200: Other net gain/loss from Form 4797
e03500: Alimony paid
e07260: Residential energy credit from Form 5695
p22250: Sch D: Net short-term capital gains/losses
e03220: Educator expenses
e07400: General business credit from Form 3800
f6251: 1 if Form 6251 (AMT) attached to return; otherwise 0
e03230: Tuition and f

In [3]:
# Create calculator from the PUF
calc_puf = Calculator(records=Records(), policy=Policy())
calc_puf.advance_to_year(2017)

You loaded data for 2009.
Tax-Calculator startup automatically extrapolated your data to 2013.


In [15]:
# Calculator the CPS
wt = pd.read_csv('../Dropbox/cps_weights.csv')
# wt.drop('SEQUENCE', inplace=True, axis=1)
# wt = wt.loc[:,:'WT2026']
wt *= 100
recs_cps = Records(data=cps,
                   weights=wt,
                   adjust_ratios=None,
                   start_year=2015)
pol_cps = Policy(start_year=2015, num_years=11)
calc_cps = Calculator(records=recs_cps, policy=pol_cps)
calc_cps.advance_to_year(2017)

In [5]:
calc_puf.calc_all()
calc_cps.calc_all()

### CPS Distribution Table

In [6]:
utils.create_distribution_table(calc_cps.records, groupby='weighted_deciles', result_type='weighted_avg')

,expanded_income,s006,c00100,num_returns_StandardDed,standard,num_returns_ItemDed,c04470,c04600,c04800,taxbc,c62100,num_returns_AMT,c09600,c05800,c07100,othertaxes,refund,iitax,payrolltax,combined
0,"1,089","16,762,987",723,"9,476,876","6,279","14,884",6,"5,022",23,1,720,0,0,1,0,0,238,-237,166,-71
1,"9,131","16,762,619","4,891","11,512,676","7,569","194,663",122,"5,820",126,9,"4,832","11,189",0,9,2,0,"2,030","-2,023",678,"-1,345"
2,"16,266","16,763,717","9,244","12,245,692","7,796","858,901",610,"6,285","1,650",157,"8,903","28,258",1,158,29,0,"3,530","-3,401","1,286","-2,115"
3,"24,244","16,763,003","17,424","13,611,231","7,834","1,868,203","1,374","7,006","5,526",587,"16,599","34,697",2,589,100,0,"4,200","-3,711","2,435","-1,276"
4,"33,864","16,763,511","27,153","13,349,979","7,564","3,019,295","2,336","7,344","12,123","1,427","25,589","29,524",4,"1,430",183,0,"3,280","-2,033","3,723","1,690"
5,"46,160","16,762,996","39,586","11,928,092","6,935","4,760,695","4,082","7,825","21,687","2,665","36,671","31,293",5,"2,671",303,0,"1,472",896,"5,237","6,132"
6,"61,998","16,763,304","55,811","10,215,270","6,370","6,536,155","6,163","8,289","35,305","4,810","51,224","27,922",4,"4,814",410,0,153,"4,252","6,969","11,221"
7,"84,003","16,763,079","77,540","7,762,990","5,313","8,996,096","8,986","9,033","54,350","8,069","70,676","29,851",5,"8,074",484,0,9,"7,581","9,464","17,044"
8,"118,863","16,761,269","110,382","5,912,745","4,303","10,846,356","12,321","10,027","83,775","13,388","100,874","18,162",3,"13,391",470,0,2,"12,920","13,400","26,319"
9,"265,852","16,765,315","251,802","3,749,302","2,791","13,015,356","17,377","9,634","222,020","50,983","237,703","1,019,951",175,"51,159",48,309,0,"51,419","21,906","73,325"


### PUF Distribution Table

In [7]:
utils.create_distribution_table(calc_puf.records, groupby='weighted_deciles', result_type='weighted_avg')

,expanded_income,s006,c00100,num_returns_StandardDed,standard,num_returns_ItemDed,c04470,c04600,c04800,taxbc,c62100,num_returns_AMT,c09600,c05800,c07100,othertaxes,refund,iitax,payrolltax,combined
0,"-6,694","17,041,816","-6,948","11,516,212","6,294","4,682",21,"4,443",58,7,"-7,360",0,0,7,0,7,137,-124,340,216
1,"8,807","17,040,415","6,002","14,010,959","7,370","53,927",27,"4,948",315,28,"5,974","14,445",0,28,1,5,743,-710,844,134
2,"15,943","17,041,505","10,265","13,678,936","8,082","312,986",210,"6,559","1,631",160,"10,122","16,510",1,161,20,7,"1,634","-1,486","1,469",-17
3,"23,438","17,044,022","15,739","14,832,468","8,336","731,409",590,"6,978","4,341",463,"15,373","35,460",2,465,93,13,"1,414","-1,028","2,105","1,077"
4,"32,816","17,041,666","24,306","14,799,151","8,250","1,706,190","1,483","7,292","10,012","1,162","23,372","19,038",2,"1,164",255,18,974,-47,"3,080","3,034"
5,"44,317","17,042,127","35,697","13,772,526","7,913","3,146,508","2,982","7,429","18,933","2,333","33,909","25,706",4,"2,337",405,30,419,"1,544","4,233","5,776"
6,"59,254","17,040,903","50,971","11,779,967","7,175","5,241,174","5,537","7,768","30,964","4,164","47,677","25,635",5,"4,169",511,45,122,"3,580","5,685","9,266"
7,"81,104","17,042,185","74,150","9,416,430","6,147","7,617,314","8,813","8,592","50,717","7,482","69,308","72,693",8,"7,490",655,68,77,"6,826","8,400","15,226"
8,"120,974","17,042,904","112,646","5,994,841","4,253","11,044,938","15,074","10,184","83,182","13,455","104,600","253,699",32,"13,487",796,105,95,"12,700","13,770","26,470"
9,"367,045","17,041,967","348,501","2,092,487","1,497","14,937,255","39,706","9,557","298,179","75,025","332,039","5,108,152","2,582","77,607","1,643","2,566",47,"78,483","24,237","102,721"


### Diagnostic Table Comparison

In [8]:
cps_diag = utils.create_diagnostic_table(calc_cps)

In [9]:
puf_diag = utils.create_diagnostic_table(calc_puf)

In [10]:
diag_data = pd.DataFrame()
diag_data['CPS'] = cps_diag[2017]
diag_data['PUF'] = puf_diag[2017]
diag_data['% Change'] = ((cps_diag[2017] / puf_diag[2017]) - 1) * 100

In [11]:
diag_data

,CPS,PUF,% Change
Returns (#m),167.6,170.4,-1.6
AGI ($b),"9,967.0","11,440.8",-12.9
Itemizers (#m),50.1,44.8,11.9
Itemized Deduction ($b),894.8,"1,268.7",-29.5
Standard Deduction Filers (#m),99.8,111.9,-10.8
Standard Deduction ($b),901.6,996.2,-9.5
Personal Exemption ($b),"1,168.1","1,175.7",-0.6
Taxable Income ($b),"7,318.8","8,492.6",-13.8
Regular Tax ($b),"1,376.3","1,777.1",-22.6
AMT Income ($b),"9,283.6","10,822.0",-14.2


In [12]:
# Total value of missing itemized deductions
in_billions = 1e-9
state = (calc_puf.records.e18400 * calc_puf.records.s006).sum()
print 'State and Local: {} ($b)'.format(round(state * in_billions, 2))
int_paid = (calc_puf.records.e19200 * calc_puf.records.s006).sum()
print 'Interest Paid: {} ($b)'.format(round(int_paid * in_billions, 2))
net_cas = (calc_puf.records.e20500 * calc_puf.records.s006).sum()
print 'Net Casualty or Theft Loss: {} ($b)'.format(round(net_cas * in_billions, 2))
print '-----------------'
print 'Total: {} ($b)'.format((round((state + int_paid + net_cas) * in_billions, 2)))

State and Local: 364.93 ($b)
Interest Paid: 379.23 ($b)
Net Casualty or Theft Loss: 3.53 ($b)
-----------------
Total: 747.69 ($b)
